In [ ]:
from telegram.client import Telegram
from dotenv import load_dotenv
import os


# JSON schema https://core.telegram.org/schema/json

In [ ]:
# Optional: enable verbose logging. Do not do this lol

import logging
# logging.root.manager.loggerDict
# Create handler
"""
handler = logging.StreamHandler()
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)

# Configure loggers
for logger_name in ['telegram', 'telegram.utils', 'telegram.tdjson', 'telegram.worker', 'telegram.client']:
    logger = logging.getLogger(logger_name)
    logger.setLevel(logging.DEBUG)
    logger.addHandler(handler)

# Optionally, set root logger to DEBUG
# logging.getLogger().setLevel(logging.DEBUG)
# logging.getLogger().addHandler(handler)
"""

In [ ]:
load_dotenv()  # take environment variables from .env.

tg = Telegram(
    api_id=os.environ['TG_API_ID'],
    api_hash=os.environ['TG_API_HASH'],
    phone=os.environ['TG_PHONE'],
    database_encryption_key=os.environ['TG_DB_KEY'],
    files_directory='/tmp/.tdlib_files/'
)

tg._tdjson.set_log_file_path(os.path.join(os.getcwd() + 'log/tdlib.log'))

def new_update_handler(update):
    # We want to process only text messages.
    print("message: ", update)
    """
    message_content = update['message']['content'].get('text', {})
    message_text = message_content.get('text', '').lower()
    is_outgoing = update['message']['is_outgoing']

    if not is_outgoing and message_text == 'ping':
        chat_id = update['message']['chat_id']
        print(f'Ping has been received from {chat_id}')
        tg.send_message(
            chat_id=chat_id,
            text='pong',
        )
    """

tg.add_global_update_handler(new_update_handler)
tg.login()


In [ ]:
# If this is the first run, the library needs to preload all chats.
# Otherwise, the message will not be sent.
all_chats = tg.get_chats()


# `tdlib` is asynchronous, so `python-telegram` always returns an `AsyncResult` object.
# You can receive a result with the `wait` method of this object.
all_chats.wait()
all_chats.update


In [ ]:
# Get info about a chat (This one is "perception training")
chat_id = -1001265830460
chat = tg.get_chat(chat_id)
chat.wait()

print(chat.update)



In [ ]:
# Send a message. This is spammy so please don't run it lol
"""
from telegram.text import Spoiler
result = tg.send_message(5694771567, Spoiler("I'm testing the Telegram API to write some code with Python."))
result.wait()
print(result.update)
"""

In [ ]:
all_chats_info = []

for chat_id in all_chats.update['chat_ids'][0:10]:
    chat = tg.get_chat(chat_id)
    chat.wait()
    print(f"{chat_id} - {chat.update['title']}")
    all_chats_info.append(chat.update)

In [ ]:
# Call an arbitrary method.
# https://core.telegram.org/tdlib/docs/classtd_1_1td__api_1_1search_chats.html

# tg._tdjson._td_set_log_verbosity_level(5)

chatSearch = tg.call_method('searchChats', {'query': "Jeremy", 'limit': 5})

chatSearch


chatSearch.wait()
chatSearch.update


# tg._tdjson._td_set_log_verbosity_level(2)

In [ ]:
recentStix = tg.call_method('getRecentStickers', {'is_attached': False})
recentStix.wait()
# recentStix.update
# Will create a very big object!!

In [ ]:
Buddies = {'Seddel': 76065323, 'Ha\'l': 5694771567 }

In [ ]:
history = tg.get_chat_history(Buddies['Seddel'], 5)
history.wait()
history.update



In [ ]:
# Most recent message
message_to_delete = history.update['messages'][0]['id']

"""
deleteMessage = tg.call_method('deleteMessages', {
    'chat_id': Buddies['Seddel'],
    'message_ids': [133637865472],
    'revoke': True
})
"""
deleteMessage = tg.delete_messages(Buddies['Seddel'], [133637865472], True)
deleteMessage.wait()

print(deleteMessage.update)




In [ ]:
# from telegram.text import PlainText

tg._tdjson._td_set_log_verbosity_level(5)
editMessage = tg.call_method('editMessageText', {
    'chat_id': 76065323,
    'message_id': 133650448384,
    'reply_markup': None,
    'input_message_content': {
        '@type': 'inputMessageText',
        'text': {
            '@type': 'formattedText',
            "text": "Maybe it's a skill issue.",
            'entities': []
        }
    }
})

editMessage.wait()
print(editMessage.update)


tg._tdjson._td_set_log_verbosity_level(2)

In [ ]:
tg.stop()  # You must call `stop` at the end of the script.